In [1]:
import requests
import pandas as pd

In [2]:
#Read all_decks file and get integer columns
df_deck = pd.read_csv("Data/all_decks.csv",nrows=1)
# Get card list
card_list = []
for column in df_deck.columns.unique():
    if 'pool_' in column:
        card = column.replace('pool_','')
        card_list.append(card)

## Get Cards per SET

In [2]:
# api-endpoint
SET = 'bro'
URL = "https://api.scryfall.com/cards/search?q=set="+SET


In [14]:
def mount_cards_df(URL, lista):
    card_attr_list = lista
    r = requests.get(url = URL)
    data = r.json()
    aux_df = pd.DataFrame()
    #Read json
    for card in data['data']:

        if len(card['color_identity']) == 0:
            color_identity = 'T'
        elif len(card['color_identity']) > 1:
            color_identity = 'M'
        else:
            color_identity = card['color_identity']

        if len(card['colors']) == 0:
            colors = 'T'
        elif len(card['colors']) > 1:
            colors = 'M'
        else:
            colors = card['colors']
        card_attr_list.append(aux_df)
        
        aux_dict = { 'name': [card['name']],
                    'rarity': [card['rarity']],
                    'color_identity': color_identity,
                    'colors': colors,
                }
        card_attr_list.append(pd.DataFrame.from_dict(aux_dict))

    if data['has_more']:
        next_page = data['next_page']
        mount_cards_df(next_page, card_attr_list)

    return card_attr_list

In [10]:
card_attr_list = mount_cards_df(URL,[])
card_attr_df = pd.concat(card_attr_list)

In [15]:
card_attr_df.to_csv('Data/BRO_cards.csv', index=False)

## Get each Card by Name

In [3]:
# api-endpoint
URL = "https://api.scryfall.com/cards/named?exact="

In [6]:
card_attr_list = []
for card_name in card_list:
    r = requests.get(url = URL+card_name)
    card = r.json()
    aux_df = pd.DataFrame()
    if len(card['color_identity']) == 0:
        color_identity = 'T'
    elif len(card['color_identity']) > 1:
        color_identity = 'M'
    else:
        color_identity = card['color_identity']

    if len(card['colors']) == 0:
        colors = 'T'
    elif len(card['colors']) > 1:
        colors = 'M'
    else:
        colors = card['colors']
    card_attr_list.append(aux_df)
    
    aux_dict = { 'name': [card['name']],
                'rarity': [card['rarity']],
                'cmc': [card['cmc']],
                'color_identity': color_identity,
                'colors': colors,
            }
    card_attr_list.append(pd.DataFrame.from_dict(aux_dict))

In [7]:
card_attr_df = pd.concat(card_attr_list)

In [8]:
card_attr_df.to_csv('Data/all_cards.csv', index=False)